In [1]:
from ppsim import Simulation, StatePlotter, HistoryPlotter
%matplotlib widget
from matplotlib import pyplot as plt
import ipywidgets as widgets

In [2]:
def make_display(sim):
    sp, hp = StatePlotter(), HistoryPlotter()
    plt.ioff()
    sim.add_snapshot(sp)
    sim.add_snapshot(hp)
    
#     def set_yscale(yscale):
#         for snap in sim.snapshots:
#             snap.ax.set_yscale(yscale)
#             snap.update()

#     yscale_button = widgets.interactive(set_yscale, yscale=widgets.ToggleButtons(options=['linear', 'symlog'], description = 'y scale:'))
    sim.layout = widgets.AppLayout(
        left_sidebar=sp.fig.canvas,
        right_sidebar=hp.fig.canvas
    )
    plt.ion()
    display(sim.layout)
    
def add_timebar(sim):
    time_bar = widgets.interactive(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1], layout=widgets.Layout(width='100%'), step=0.01))
    sim.layout.footer=time_bar

In [3]:
a, b, u = 'A', 'B', 'U'
approximate_majority = {
    (a,b):(u,u),
    (b,a):(u,u),
    (a,u):(a,a),
    (u,a):(a,a),
    (b,u):(b,b),
    (u,b):(b,b)
}

def self_stabilizing_clock(a, b, m, p=1):
    # states a, b in (0, ..., m-1, 'N')
    if a in range(m) and b in range(m):
        # drip reaction
        if a == b:
            return {((a + 1) % m, b): p}
        # epidemic reaction
        elif (a + 1) % m == b:
            return b, b
        elif (b + 1) % m == a:
            return a, a
        # clipping reaction
        else:
            return 'N', 'N'
    # recovery epidemic reactions
    elif a == 'N':
        return b, b
    elif b == 'N':
        return a, a
    
m = 12
n = 10 ** 8
sim = Simulation({i: n // m for i in range(m)}, self_stabilizing_clock, m=m)

# n = 10 ** 8
# config = {a: n // 2, b: n // 2}
# amsim = Simulation(config, approximate_majority)


make_display(sim)

# sp, hp = StatePlotter(), HistoryPlotter()
# plt.ioff()
# sim.add_snapshot(sp)
# sim.add_snapshot(hp)
# plt.ion()
# layout = widgets.AppLayout(
#     left_sidebar=sp.fig.canvas,
#     right_sidebar=hp.fig.canvas
# )
# display(layout)

AppLayout(children=(Canvas(layout=Layout(grid_area='left-sidebar'), toolbar=Toolbar(toolitems=[('Home', 'Reset…

In [4]:
sim.run(400)

<class 'ppsim.ppsim.StatePlotter'> took 0.0549987999999999 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.05847879999999961 seconds.


Canvas(layout=Layout(grid_area='right-sidebar'), toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'h…

<class 'ppsim.ppsim.HistoryPlotter'> took 0.19475470000000072 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.0476321999999989 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.04932379999999981 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.03919379999999961 seconds.
<class 'ppsim.ppsim.HistoryPlotter'> took 0.17069920000000138 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.03414839999999941 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.055574299999999965 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.05144779999999827 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.04085720000000137 seconds.
<class 'ppsim.ppsim.HistoryPlotter'> took 0.17932770000000176 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.05309689999999989 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.04673090000000002 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.048603199999998736 seconds.
<class 'ppsim.ppsim.StatePlotter'> took 0.036972900000002085 seconds.
<class 'ppsim.ppsim.Histor

In [5]:
add_timebar(sim)

In [8]:
plt.ioff()
sim.run(100., time_step=0.1)
time_bar = widgets.interactive(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1], layout=widgets.Layout(width='100%'), step=0.01))
layout.footer=time_bar

def set_yscale(yscale):
    for snap in sim.snapshots:
        snap.ax.set_yscale(yscale)
        snap.update()
    
yscale_button = widgets.interactive(set_yscale, yscale=widgets.ToggleButtons(options=['linear', 'symlog'], description = 'y scale:'))
layout.header=yscale_button

In [7]:
from math import ceil, floor

def discrete_averaging(a, b):
    avg = (a + b) / 2
    return floor(avg), ceil(avg)

In [27]:
n = 10 ** 10
sim = Simulation({0: n // 2, 99: n // 2}, discrete_averaging)
plt.ioff()
sim.add_snapshot(StatePlotter())
sim.snapshots[0].update_interval=1
plt.ion()
layout = widgets.AppLayout(
    center=sim.snapshots[0].fig.canvas
)
layout

AppLayout(children=(Canvas(layout=Layout(grid_area='center'), toolbar=Toolbar(toolitems=[('Home', 'Reset origi…

In [28]:
xlabels = list(range(100))
for i in range(100):
    if i % 5 != 0:
        xlabels[i] = ''
sim.snapshots[0].ax.set_xticklabels(xlabels)
%lprun -f sim.run sim.run(time_step=0.05, snapshot_rate=5)

Timer unit: 1e-07 s

Total time: 84.6934 s
File: C:\Users\Eric\PycharmProjects\ppsim\ppsim\ppsim.py
Function: run at line 274

Line #      Hits         Time  Per Hit   % Time  Line Contents
   274                                               def run(self, run_time=None, time_step=1., snapshot_rate=0.1):
   275                                                   """Runs the simulation for a fixed amount of time, or until it becomes silent.
   276                                           
   277                                                   Args:
   278                                                       run_time (Optional[Union[float, int]]): The length to run the simulation.
   279                                                           An int is interpreted to be a number of interaction steps, while a float
   280                                                           is interpreted to be units of parallel time (n steps). Defaults to None.
   281                            

In [29]:
time_bar = widgets.interactive(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1], layout=widgets.Layout(width='100%'), step=0.01))
layout.footer=time_bar

def set_yscale(yscale):
    sim.snapshots[0].ax.set_yscale(yscale)
    
yscale_button = widgets.interactive(set_yscale, yscale=widgets.ToggleButtons(options=['linear', 'symlog'], description = 'y scale:'))
layout.header=yscale_button

In [35]:

display(sim.snapshots[0].fig.canvas)
widgets.interact(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='time', max=24.928493), Output()), _dom_classes=('wid…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [43]:
sim.set_snapshot_time(0)

In [45]:
import numpy as np
np.searchsorted(sim.times, 0)

0

In [28]:
sim.snapshots[0].ax.set_yscale('symlog')

In [41]:
xlabels = list(range(100))
for i in range(100):
    if i % 5 != 0:
        xlabels[i] = ''
sim.snapshots[0].ax.set_xticklabels(xlabels)

[Text(0, 0, '0'),
 Text(1, 0, ''),
 Text(2, 0, ''),
 Text(3, 0, ''),
 Text(4, 0, ''),
 Text(5, 0, '5'),
 Text(6, 0, ''),
 Text(7, 0, ''),
 Text(8, 0, ''),
 Text(9, 0, ''),
 Text(10, 0, '10'),
 Text(11, 0, ''),
 Text(12, 0, ''),
 Text(13, 0, ''),
 Text(14, 0, ''),
 Text(15, 0, '15'),
 Text(16, 0, ''),
 Text(17, 0, ''),
 Text(18, 0, ''),
 Text(19, 0, ''),
 Text(20, 0, '20'),
 Text(21, 0, ''),
 Text(22, 0, ''),
 Text(23, 0, ''),
 Text(24, 0, ''),
 Text(25, 0, '25'),
 Text(26, 0, ''),
 Text(27, 0, ''),
 Text(28, 0, ''),
 Text(29, 0, ''),
 Text(30, 0, '30'),
 Text(31, 0, ''),
 Text(32, 0, ''),
 Text(33, 0, ''),
 Text(34, 0, ''),
 Text(35, 0, '35'),
 Text(36, 0, ''),
 Text(37, 0, ''),
 Text(38, 0, ''),
 Text(39, 0, ''),
 Text(40, 0, '40'),
 Text(41, 0, ''),
 Text(42, 0, ''),
 Text(43, 0, ''),
 Text(44, 0, ''),
 Text(45, 0, '45'),
 Text(46, 0, ''),
 Text(47, 0, ''),
 Text(48, 0, ''),
 Text(49, 0, ''),
 Text(50, 0, '50'),
 Text(51, 0, ''),
 Text(52, 0, ''),
 Text(53, 0, ''),
 Text(54, 0, ''),


In [46]:
def self_stabilizing_clock(a, b, m, p=1):
    # states a, b in (0, ..., m-1, 'N')
    if a in range(m) and b in range(m):
        # drip reaction
        if a == b:
            return {((a + 1) % m, b): p}
        # epidemic reaction
        elif (a + 1) % m == b:
            return b, b
        elif (b + 1) % m == a:
            return a, a
        # clipping reaction
        else:
            return 'N', 'N'
    # recovery epidemic reactions
    elif a == 'N':
        return b, b
    elif b == 'N':
        return a, a
    
m = 12
n = 10 ** 8
sim = Simulation({0:n}, self_stabilizing_clock, m=m)
sim.add_snapshot(StatePlotter())
sim.add_snapshot(HistoryPlotter())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
widgets.interact(sim.set_snapshot_time, time=widgets.FloatSlider(min=sim.times[0], max=sim.times[-1]))

interactive(children=(FloatSlider(value=0.0, description='time', max=100.02578919), Output()), _dom_classes=('…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [2]:
from typing import NamedTuple
from math import log2, ceil

class Agent(NamedTuple):
    value: int
    x: int
    y: int
    output: int
        
def initial_agent(opinion, n):
    L = 2 ** ceil(log2(n))
    if opinion.lower() == 'a':
        return Agent(value = L, x = L, y = 0, output = 1)
    elif opinion.lower() == 'b':
        return Agent(value = -L, x = 0, y = L, output = -1)
    else:
        raise ValueError('opinion must be "a" or "b"')
        
def reduce(u, v):
    if u == v:
        return 0, 0
    elif u == 2 * v:
        return u - v, 0
    elif 2 * u == v:
        return 0, v - u
    else:
        return u, v
    
def cancel(x1, y1, x2, y2):
    x1o, y2o = reduce(x1, y2)
    x2o, y1o = reduce(x2, y1)
    return x1o, y1o, x2o, y2o

def join(x1, y1, x2, y2):
    if x1 - y1 > 0 and x2 - y2 > 0 and y1 == y2:
        y1o, y2o = y1 + y2, 0
    else:
        y1o, y2o = y1, y2
    if x1 - y1 < 0 and x2 - y2 < 0 and x1 == x2:
        x1o, x2o = x1 + x2, 0
    else:
        x1o, x2o = x1, x2
    return x1o, y1o, x2o, y2o

def split(x1, y1, x2, y2):
    if (x1 - y1 > 0 or x2 - y2 > 0) and max(x1, x2) > 1 and min(x1, x2) == 0:
        x1o = x2o = max(x1, x2) // 2
    else:
        x1o, x2o = x1, x2
    if (x1 - y1 < 0 or x2 - y2 < 0) and max(y1, y2) > 1 and min(y1, y2) == 0:
        y1o = y2o = max(y1, y2) // 2
    else:
        y1o, y2o = y1, y2
    return x1o, y1o, x2o, y2o
        
def normalize(x, y, v):
    x1, y1 = reduce(x, y)
    if x1 == y1 == 0:
        if v >= 0:
            output = +1
        else:
            output = -1
    else:
        if x1 > y1:
            output = +1
        else:
            output = -1
    return Agent(x = x1, y = y1, output = output, value = x1 - y1)
    
def split_join_majority(a, b):
    if a.x == a.y == b.x == b.y == 0:
        return a, b
    else:
        x1, y1, x2, y2 = split(*join(*cancel(a.x, a.y, b.x, b.y)))
    return normalize(x1, y1, x2 - y2), normalize(x2, y2, x1 - y1)

In [14]:
n = 10 ** 8
sim = Simulation({initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2}, split_join_majority)
# make_display(sim)

In [7]:
def converged(config):
    outputs = {state.output for state in config.keys()}
    return len(outputs) == 1

{1}

In [15]:
sim.run(converged)
sim.config_dict

{Agent(value=0, x=0, y=0, output=1): 505025,
 Agent(value=1, x=1, y=0, output=1): 86885291,
 Agent(value=2, x=2, y=0, output=1): 8121483,
 Agent(value=3, x=4, y=1, output=1): 15,
 Agent(value=4, x=4, y=0, output=1): 3034289,
 Agent(value=6, x=8, y=2, output=1): 16,
 Agent(value=7, x=8, y=1, output=1): 4,
 Agent(value=8, x=8, y=0, output=1): 1017834,
 Agent(value=12, x=16, y=4, output=1): 142,
 Agent(value=14, x=16, y=2, output=1): 5,
 Agent(value=15, x=16, y=1, output=1): 1,
 Agent(value=16, x=16, y=0, output=1): 313711,
 Agent(value=24, x=32, y=8, output=1): 218,
 Agent(value=28, x=32, y=4, output=1): 56,
 Agent(value=30, x=32, y=2, output=1): 2,
 Agent(value=32, x=32, y=0, output=1): 89684,
 Agent(value=48, x=64, y=16, output=1): 178,
 Agent(value=56, x=64, y=8, output=1): 59,
 Agent(value=60, x=64, y=4, output=1): 16,
 Agent(value=62, x=64, y=2, output=1): 1,
 Agent(value=63, x=64, y=1, output=1): 1,
 Agent(value=64, x=64, y=0, output=1): 24022,
 Agent(value=96, x=128, y=32, output=

In [16]:
sim.run()
sim.config_dict

{Agent(value=1, x=1, y=0, output=1): 87536220,
 Agent(value=2, x=2, y=0, output=1): 8038478,
 Agent(value=4, x=4, y=0, output=1): 2994060,
 Agent(value=8, x=8, y=0, output=1): 1002449,
 Agent(value=16, x=16, y=0, output=1): 308867,
 Agent(value=32, x=32, y=0, output=1): 88461,
 Agent(value=64, x=64, y=0, output=1): 23754,
 Agent(value=128, x=128, y=0, output=1): 6082,
 Agent(value=256, x=256, y=0, output=1): 1295,
 Agent(value=512, x=512, y=0, output=1): 282,
 Agent(value=1024, x=1024, y=0, output=1): 46,
 Agent(value=2048, x=2048, y=0, output=1): 7}

{Agent(value=1, x=1, y=0, output=1): 975538,
 Agent(value=2, x=2, y=0, output=1): 17883,
 Agent(value=4, x=4, y=0, output=1): 4944,
 Agent(value=8, x=8, y=0, output=1): 1273,
 Agent(value=16, x=16, y=0, output=1): 303,
 Agent(value=32, x=32, y=0, output=1): 49,
 Agent(value=64, x=64, y=0, output=1): 10,
 Agent(value=256, x=256, y=0, output=1): 1}

In [58]:
sim.history.groupby(axis = 1, level='output')

In [91]:
sim.history.groupby(axis = 1, level = ["y"]).get_group(0).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [107]:
def stability_check(config_dict):
    return bool

{0: [(-1048576, 0, 1048576, -1)], 1: [(-1048574, 2, 1048576, -1)]}

In [112]:
config = sim.history.iloc[0]
config.groupby(level = ["x", "output"]).indices

{(0,
  -1): array([  0,  19,  38,  56,  73,  89, 104, 118, 131, 143, 154, 164, 173,
        181, 188, 194, 199, 203, 206, 208, 209, 210], dtype=int64),
 (0, 1): array([211], dtype=int64),
 (1,
  -1): array([ 20,  39,  57,  74,  90, 105, 119, 132, 144, 155, 165, 174, 182,
        189, 195, 200, 204, 207], dtype=int64),
 (1, 1): array([212], dtype=int64),
 (2,
  -1): array([  1,  21,  40,  58,  75,  91, 106, 120, 133, 145, 156, 166, 175,
        183, 190, 196, 201, 205], dtype=int64),
 (2, 1): array([213], dtype=int64),
 (4,
  -1): array([  2,  22,  41,  59,  76,  92, 107, 121, 134, 146, 157, 167, 176,
        184, 191, 197, 202], dtype=int64),
 (4, 1): array([214, 215], dtype=int64),
 (8,
  -1): array([  3,  23,  42,  60,  77,  93, 108, 122, 135, 147, 158, 168, 177,
        185, 192, 198], dtype=int64),
 (8, 1): array([216, 217, 218], dtype=int64),
 (16,
  -1): array([  4,  24,  43,  61,  78,  94, 109, 123, 136, 148, 159, 169, 178,
        186, 193], dtype=int64),
 (16, 1): array([219, 

In [146]:
import seaborn as sns
import pandas as pd
fig, ax = plt.subplots()
df = sim.column_names.to_frame().convert_dtypes()
df['count'] = sim.configs[40]
df = pd.concat([df.xs(-1, level="bias"), df.xs(1, level="bias")])
sns.barplot(x="exponent", y="count", hue="bias", data=df, estimator=sum, ci=None, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='exponent', ylabel='count'>

In [127]:
from majority_main_averaging import *

In [215]:
n = 10 ** 10
init_dist = {make_agent('A'): n // 4 + 1, make_agent('B'): n // 4 - 1, make_agent('C'): n // 2}
sim = ppsim.Simulation(init_dist, majority_main_averaging,
                       L=int(np.log2(n))+1, k=3, p=1/4)

In [216]:
sim.add_snapshot(BiasPlotter())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [260]:
from natsort import natsorted
sim.history
['bias', 'role']

# map
def state_map(state):
    if state.role == 'Clock':
        if state.minute < 20:
            return 'early clock'
        else:
            return 'late clock'
    else:
        if state.bias == -1:
            return 'opinion B'
        if state.bias == 1:
            return 'opinion A'
        if state.bias == 0:
            return 'unbiased'
        
def state_map(state):
    if state.x == 0 and state.y != 0:
        return 'pure y'
    elif state.y == 0 and state.x != 0:
        return 'pure x'
    elif state.x == state.y == 0:
        return 'zero'
    else:
        return 'mixed'

categories = natsorted(set([state_map(state) for state in sim.state_list if state_map(state) is not None]))
categories_dict = {j: i for i, j in enumerate(categories)}
matrix = np.zeros((len(categories), len(sim.state_list)), dtype=np.int64)
for i, state in enumerate(sim.state_list):
    m = state_map(state)
    if m is not None:
        matrix[categories_dict[m], i] += 1

plt.figure()
sns.barplot(x = categories, y = np.matmul(sim.config_array, matrix.T))
matrix.shape

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(4, 422)

In [261]:
pd.DataFrame(np.matmul(sim.history.to_numpy(), matrix.T), columns = categories, index = sim.history.index).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [256]:
sim.run(100.)

KeyboardInterrupt: 

In [205]:
from ppsim import Snapshot
class BiasPlotter(Snapshot):

    def initialize(self):
        self.fig, self.ax = plt.subplots()

    def update(self, index=None):
        super().update(index)
        self.ax.clear()
        self.simulation.history.T.groupby(level=('bias','role')).sum().T.plot(ax = self.ax)
        if index is not None:
            self.ax.axvline(x=self.time)
        self.fig.canvas.draw()

In [226]:
pd.concat([sim.history.xs(-1, level='bias', axis = 1, drop_level=False), sim.history.xs(1, level='bias', axis = 1, drop_level=False)])

role      Main                                               ...            \
minute     NaN                                               ...             
hour       NaN                                               ...             
exponent   -34  -33  -32  -31  -30  -29  -28  -27  -26  -25  ...  -9   -8    
bias        -1   -1   -1   -1   -1   -1   -1   -1   -1   -1  ...    1    1   
time                                                         ...             
0.000000     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
1.000002     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
2.000055     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
3.000166     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
4.000334     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
5.000746     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
6.001314     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
7.001899     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
8.002428     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
9.003060     0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
10.003687    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
11.004303    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
12.005088    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
13.005922    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
14.006591    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
15.007272    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
16.007972    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
17.008649    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
18.009434    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
19.010201    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
20.010926    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
21.011788    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
22.012601    0    0    0    0    0    0    0    0    0    0  ...  NaN  NaN   
0.000000   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
1.000002   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
2.000055   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
3.000166   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
4.000334   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
5.000746   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
6.001314   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
7.001899   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
8.002428   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
9.003060   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
10.003687  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
11.004303  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
12.005088  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
13.005922  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
14.006591  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
15.007272  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
16.007972  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
17.008649  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
18.009434  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
19.010201  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
20.010926  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
21.011788  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...    0    0   
22.012601  NaN  NaN  N

In [27]:
h.iloc[1].groupby(level = ['x', 'output']).sum()

x     output
0     -1        382
       1        160
1     -1          0
       1          0
2     -1          0
       1          0
4     -1          0
       1          0
8     -1          2
       1          0
16    -1          2
       1          3
32    -1          6
       1          6
64    -1         12
       1          7
128   -1         26
       1         35
256   -1          0
       1         73
512    1        143
1024   1        144
Name: 1.0999000999000998, dtype: int64

In [8]:
h = sim.history

value,-1024,-1022,-1020,-1016,-1008,-992,-960,-896,-768,-512,...,512,768,896,960,992,1008,1016,1020,1022,1024
x,0,2,4,8,16,32,64,128,256,0,...,512,1024,1024,1024,1024,1024,1024,1024,1024,1024
y,1024,1024,1024,1024,1024,1024,1024,1024,1024,512,...,0,256,128,64,32,16,8,4,2,0
output,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1
time,,,,,,,,,,,,,,,,,,,,,
0.000000,500,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,501
1.099900,133,0,0,0,0,0,0,2,0,127,...,99,0,1,0,0,0,0,0,0,143
2.329670,29,0,0,0,1,0,0,0,0,77,...,38,0,4,2,0,0,0,0,0,41
3.687313,12,0,0,0,0,1,0,0,0,28,...,24,0,3,1,1,1,0,0,0,7
5.045954,5,0,0,0,0,0,1,0,0,5,...,12,0,0,1,0,1,0,0,0,1
6.388611,1,0,0,0,0,1,0,0,0,4,...,4,0,0,0,0,0,0,0,0,0


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

d = {'n':[], 'silence time':[]}
num_trials = 1
for n in [int(n) for n in np.geomspace(10 ** 2,  10 ** 4, 20)]:
    print(f'n = {n}')
    sim = Simulation({initial_agent('a',n): n // 2 + 1, initial_agent('b', n): n // 2}, split_join_majority)
    print(sim)
    print(sim.sample_silence_time())
#     for _ in range(num_trials):
#         d['n'].append(n)
#         d['silence time'].append(sim.sample_silence_time())

# df = pd.DataFrame(data=d)

# sns.lineplot(x="n", y="silence time", data=df)

n = 100


NameError: name 'Simulation' is not defined

In [51]:
split_join_majority(a, c)

(Agent(x=16, y=0, output=1, value=16), Agent(x=16, y=0, output=1, value=16))

In [56]:
sim = Simulation({a: 100, b: 100}, split_join_majority)
sim.state_list

[Agent(x=0, y=0, output=-1, value=0),
 Agent(x=0, y=0, output=1, value=0),
 Agent(x=0, y=1, output=-1, value=-1),
 Agent(x=0, y=2, output=-1, value=-2),
 Agent(x=0, y=4, output=-1, value=-4),
 Agent(x=0, y=8, output=-1, value=-8),
 Agent(x=0, y=16, output=-1, value=-16),
 Agent(x=0, y=32, output=-1, value=-32),
 Agent(x=1, y=0, output=1, value=1),
 Agent(x=1, y=4, output=-1, value=-3),
 Agent(x=1, y=8, output=-1, value=-7),
 Agent(x=1, y=16, output=-1, value=-15),
 Agent(x=2, y=0, output=1, value=2),
 Agent(x=2, y=8, output=-1, value=-6),
 Agent(x=2, y=16, output=-1, value=-14),
 Agent(x=2, y=32, output=-1, value=-30),
 Agent(x=4, y=0, output=1, value=4),
 Agent(x=4, y=1, output=1, value=3),
 Agent(x=4, y=16, output=-1, value=-12),
 Agent(x=4, y=32, output=-1, value=-28),
 Agent(x=8, y=0, output=1, value=8),
 Agent(x=8, y=1, output=1, value=7),
 Agent(x=8, y=2, output=1, value=6),
 Agent(x=8, y=32, output=-1, value=-24),
 Agent(x=16, y=0, output=1, value=16),
 Agent(x=16, y=1, output=1

In [55]:
sorted([state.value for state in sim.state_list])

[-32,
 -30,
 -28,
 -24,
 -16,
 -15,
 -14,
 -12,
 -8,
 -7,
 -6,
 -4,
 -3,
 -2,
 -1,
 0,
 0,
 1,
 2,
 3,
 4,
 6,
 7,
 8,
 12,
 14,
 15,
 16,
 24,
 28,
 30,
 32]